In [1]:
# ! pip install opencv-python

In [1]:
from platform import python_version

print(python_version())

3.8.11


In [2]:
################### Crop image 파일에 들어있는 이미지 한 번에 저장하기 위한 것 + Sobel operator랑 Histogram eq 자리변경 (0218)
# 기존은 선 histo 후 sobel 여기서는 선 sobel 후 histo
import cv2 as cv
import os
import glob
import numpy as np
from PIL import Image, ImageOps
import math
from matplotlib import pyplot as plt
from matplotlib.pyplot import imshow
from scipy import signal

In [3]:
# conditions에 대하여

# fine-tune 이미지
#camera_list = ['5','6','7','8','9','16'] #4, 정면은 7 (C)
#light_list = ['2','3','4','5', '8','9', '12','13', '19','20', '22','23', '25','26'] #2 (L)
#accessory_list =['1'] #1 (S)
#expression_list = ['1','2'] #3 (E)

# test 이미지
camera_list = ['4','7','10'] #4, 정면은 7
light_list = ['1'] #2
accessory_list =['1'] #1
expression_list = ['1','2','3'] #320


# grayscale, pixel에 대하여

# fine-tune 이미지
#grayscale = [8]
#pixel = [64] # 128

# test 이미지
grayscale = [2,4,6,8,16]
pixel = [16,24,32,64,128]

In [4]:
len(camera_list)*len(light_list)*len(accessory_list)*len(expression_list)

9

In [5]:
# path = 'C:\\Users\\user\\Desktop\\Middle_Resolution_177_unzipped_parcropped_128_removed'
# outpath = 'C:\\Users\\user\\Desktop\\Middle_Resolution_177_unzipped_parcropped_128_elec'
path = 'C:\\Users\\user\\Desktop\\Middle_Resolution_137_unzipped_parcropped_128_removed_train'
outpath = 'C:\\Users\\user\\Desktop\\Middle_Resolution_137_unzipped_parcropped_128_elec'

folderPaths = [os.path.join(path, folder_name) for folder_name in os.listdir(path)] # path안에 있는 모든 사람 번호 폴더들

#a=0
for folderPath in folderPaths: # 모든 사람 폴더들에 대하여
    #while a<1: # 시범 
    print(folderPath)

    folder_name = os.path.basename(folderPath) # 사람 숫자
    os.makedirs(outpath+'\\'+folder_name, exist_ok=True) # 사람 숫자 폴더 만들기
    person_outpath = outpath + '\\' + folder_name # 만들어진 사람 숫자 폴더
    filePaths = [os.path.join(folderPath, file_name) for file_name in os.listdir(folderPath)] # 한 사람 폴더에 들어있는 배경 제거한 이미지들

    # # Parameter combination 파일 만들기
    # for i in pixel:
    #     for j in grayscale:
    #         #os.makedirs(person_outpath +'\\%dPIX_%dGS'%(i,j),exist_ok=True)
        
    for filePath in filePaths: # 모든 파일들에 대하여
        
        # 파일들 중에서 관심있는 parameter들에 대해서
        file_name  = os.path.basename(filePath)
        
        sl_split = file_name.split('L')
        file_name2 = sl_split[-1]
        e_split = file_name2.split('E')
        file_name3 = e_split[-1]
        c_split = file_name3.split('C')
        file_name4 = c_split[-1]
        dot_split = file_name4.split('.')

        if (str(sl_split[0][-1]) in accessory_list) and (str(e_split[0]) in light_list) and (str(c_split[0][-1]) in expression_list) and (str(dot_split[0]) in camera_list):
            
            img = cv.imread(filePath, cv.IMREAD_GRAYSCALE) # 1) original image

            #  ##### Image sharpening(enhancing) using sobel operator

            #  img_sobel_x = cv.Sobel(img, cv.CV_64F,1,0,ksize=3)
            #  img_sobel_x = cv.convertScaleAbs(img_sobel_x)

            #  img_sobel_y = cv.Sobel(img, cv.CV_64F,0,1,ksize=3)
            #  img_sobel_y = cv.convertScaleAbs(img_sobel_y)

            #  img_sobel = cv.addWeighted(img_sobel_x,1,img_sobel_y,1,0) 

            #  sobel_subtraction = cv.subtract(img, img_sobel)    


            ##### Histogram equalization 
            #  hist_equalize_img = cv.equalizeHist(sobel_subtraction)
            hist_equalize_img = cv.equalizeHist(img) # 2) It improves the contrast in an image by stretching out the intensity range.

            for px in pixel:
                for gr in grayscale:   


                ##### 3) Down-sampling Step 1: Gray-scales
                    gray_img = np.array(hist_equalize_img)
                    c = round(255/gr)

                    if gr == 2:
                        gray_img[(gray_img>=0) & (gray_img<c)] = 0 # black
                        gray_img[(gray_img>=c) & (gray_img<=255)] = 255 # white
                        
                    else:
                        for a in range(0, 256, c):
                            d = a+c
                            if d > 255: # 혹시라도 RGB 범위를 벗어나면
                                d = 255

                            if a == 0: # 처음
                                gray_img[(gray_img>=0) & (gray_img<d)] = 0
                            elif (255 - a) < c: # 마지막
                                gray_img[(gray_img>=a) & (gray_img<d)] = 255
                            else:
                                gray_img[(gray_img>=a) & (gray_img<d)] = round((a+d)/2)

                    ##### 4) Down-sampling Step 2: Pixelization
                    pixel_img = cv.resize(gray_img, dsize=(px,px), interpolation=cv.INTER_NEAREST) # (px, px), INTER_AREA: 이미지 축소시 많이 사용        
                    largest_pixel_size = max(pixel) # 나중 크기 일정
                    #pixel_img = cv.resize(shrink_img, dsize=(px, px), interpolation=cv.INTER_NEAREST) # INTER_LINEAR, INTER_CUBIC: 이미지 확대시 많이 사용 /
                    #pixel_img2 = cv.resize(pixel_img, dsize=(largest_pixel_size, largest_pixel_size), interpolation=cv.INTER_NEAREST) # 크기 일정하게 (모든 픽셀 파라미터마다px,px), interpolation=cv.INTER_LINEAR) # (px, px), INTER_AREA: 이미지 축소시 많이 사용        

                ##### 5) Down-sampling Step 3: Phosphene     
                    k_size =  (largest_pixel_size // px) * 5
                    kernel1d = signal.gaussian(k_size, std=k_size/4).reshape(k_size, 1)
                    kernel2d = np.outer(kernel1d, kernel1d)
                    g_kernel_img = kernel2d

                    new_img = np.zeros((k_size*pixel_img.shape[0], k_size*pixel_img.shape[1]))
                    for i in range(0, pixel_img.shape[0]):
                        for j in range(0, pixel_img.shape[1]):
                            new_img[k_size*i:k_size*(i+1), k_size*j:k_size*(j+1)] = pixel_img[i, j] * g_kernel_img
                    new_img1 = cv.resize(new_img, dsize=(128, 128), interpolation=cv.INTER_AREA)

                    final_img = new_img1
                    cv.imwrite(person_outpath
                        + '\\'
                        + '%dPIX_%dGS_'%(px, gr)
                        + os.path.basename(file_name),
                        final_img) 

                    # 시각화
                    #n = 7 # 몇 개?
                    #fig, ax = plt.subplots(1, n, figsize=(5*n, 5)) # 가로, 세로
                    #ax[0].imshow(np.asarray(img), cmap='gray')
                    #ax[1].imshow(np.asarray(hist_equalize_img), cmap='gray')
                    #ax[2].imshow(np.asarray(gray_img), cmap='gray')
                    #ax[3].imshow(np.asarray(pixel_img), cmap='gray')
                    #ax[4].imshow(np.asarray(kernel2d), cmap='gray') 
                    #ax[5].imshow(new_img, cmap='gray')
                    #ax[6].imshow(np.asarray(new_img2), cmap='gray')
                    #plt.title('{}_{}'.format(px, gr))  
                    #plt.show()
#a=a+1

C:\Users\user\Desktop\Middle_Resolution_137_unzipped_parcropped_128_removed_train\19062811
C:\Users\user\Desktop\Middle_Resolution_137_unzipped_parcropped_128_removed_train\19062831
C:\Users\user\Desktop\Middle_Resolution_137_unzipped_parcropped_128_removed_train\19062832
C:\Users\user\Desktop\Middle_Resolution_137_unzipped_parcropped_128_removed_train\19070121
C:\Users\user\Desktop\Middle_Resolution_137_unzipped_parcropped_128_removed_train\19070221
C:\Users\user\Desktop\Middle_Resolution_137_unzipped_parcropped_128_removed_train\19070231
C:\Users\user\Desktop\Middle_Resolution_137_unzipped_parcropped_128_removed_train\19070311
C:\Users\user\Desktop\Middle_Resolution_137_unzipped_parcropped_128_removed_train\19070312
C:\Users\user\Desktop\Middle_Resolution_137_unzipped_parcropped_128_removed_train\19070321
C:\Users\user\Desktop\Middle_Resolution_137_unzipped_parcropped_128_removed_train\19070322
C:\Users\user\Desktop\Middle_Resolution_137_unzipped_parcropped_128_removed_train\19070331